<a href="https://colab.research.google.com/github/Suvana-Rohanlal/Race_predictor/blob/main/race_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastf1 pandas numpy scikit-learn

In [ ]:
import fastf1
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
# Enable FastF1 caching
# fastf1.Cache.enable_cache("f1_cache")

# Load FastF1 2024 Australian GP race session
session_2024 = fastf1.get_session(2024, 3, "R")
# print(session_2024.event)
session_2024.load()

In [ ]:
# Extract lap times
laps_2024 = session_2024.laps[["Driver", "LapTime"]].copy()
# Drop NA rows
laps_2024.dropna(subset=["LapTime"], inplace=True)
# Create a column with the laptime in seconds
laps_2024["LapTime (s)"] = laps_2024["LapTime"].dt.total_seconds()


In [ ]:
driver_mapping = { "Lando Norris": "NOR", "Oscar Piastri": "PIA", "Max Verstappen": "VER", "George Russell": "RUS",
                  "Yuki Tsunoda": "TSU", "Alexander Albon": "ALB", "Charles Leclerc": "LEC", "Lewis Hamilton": "HAM",
                   "Pierre Gasly": "GAS", "Carlos Sainz": "SAI", "Lance Stroll": "STR", "Fernando Alonso": "ALO" }

# Compute average race pace per driver from 2024
driver_avg_times = laps_2024.groupby("Driver")["LapTime (s)"].mean().reset_index()
driver_avg_times.rename(columns={"LapTime (s)": "AvgRacePace_2024"}, inplace=True)


# Map FastF1 codes back to full names
driver_mapping_rev = {v: k for k, v in driver_mapping.items()}  # invert your dict
driver_avg_times["Driver"] = driver_avg_times["Driver"].map(driver_mapping_rev)

# Keep only the 2025 drivers we care about
drivers_2025 = list(driver_mapping.keys())
avg_racepace_2024 = driver_avg_times[driver_avg_times["Driver"].isin(drivers_2025)].reset_index(drop=True)

# Final DataFrame in same style as qualifying_2025
racepace_2024 = pd.DataFrame({
    "Driver": avg_racepace_2024["Driver"],
    "DriverCode": avg_racepace_2024["Driver"].map(driver_mapping),   # add 3-letter code
    "AvgRacePace_2024 (s)": avg_racepace_2024["AvgRacePace_2024"]
})

# Merge with 2024 averages
merged_data = racepace_2024.merge(laps_2024, left_on="DriverCode", right_on="Driver", how="left")
print(merged_data)

In [ ]:
# # 2025 Qualifying Data
# qualifying_2025 = pd.DataFrame({
#     "Driver": ["Lando Norris", "Oscar Piastri", "Max Verstappen", "George Russell", "Yuki Tsunoda",
#                "Alexander Albon", "Charles Leclerc", "Lewis Hamilton", "Pierre Gasly", "Carlos Sainz", "Fernando Alonso", "Lance Stroll"],
#     "QualifyingTime (s)": [75.096, 75.180, 75.481, 75.546, 75.670,
#                            75.737, 75.755, 75.973, 75.980, 76.062, 76.4, 76.5]
# })

# # Map full names to FastF1 3-letter codes
# driver_mapping = {
#     "Lando Norris": "NOR", "Oscar Piastri": "PIA", "Max Verstappen": "VER", "George Russell": "RUS",
#     "Yuki Tsunoda": "TSU", "Alexander Albon": "ALB", "Charles Leclerc": "LEC", "Lewis Hamilton": "HAM",
#     "Pierre Gasly": "GAS", "Carlos Sainz": "SAI", "Lance Stroll": "STR", "Fernando Alonso": "ALO"
# }

# qualifying_2025["DriverCode"] = qualifying_2025["Driver"].map(driver_mapping)

# # Merge 2025 Qualifying Data with 2024 Race Data
# merged_data = qualifying_2025.merge(laps_2024, left_on="DriverCode", right_on="Driver")
# print(merged_data)

In [ ]:

# Use only average 2024 pace as feature
X = merged_data[["AvgRacePace_2024 (s)"]]
y = merged_data["LapTime (s)"]

# # Use only "QualifyingTime (s)" as a feature
# X = merged_data[["QualifyingTime (s)"]]
# y = merged_data["LapTime (s)"]

if X.shape[0] == 0:
    raise ValueError("Dataset is empty after preprocessing. Check data sources!")

In [ ]:
# Train Gradient Boosting Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=39)
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=39)
model.fit(X_train, y_train)


In [ ]:
# Predict 2025 race pace using 2024 averages
predicted_lap_times = model.predict(racepace_2024[["AvgRacePace_2024 (s)"]])
racepace_2024["PredictedRaceTime (s)"] = predicted_lap_times

# # Predict using 2025 qualifying times
# predicted_lap_times = model.predict(qualifying_2025[["QualifyingTime (s)"]])
# qualifying_2025["PredictedRaceTime (s)"] = predicted_lap_times

In [ ]:
# Rank drivers by predicted race time
racepace_2024 = racepace_2024.sort_values(by="PredictedRaceTime (s)")

# Print final predictions
print("\n🏁 Predicted 2025 Australia GP Winner 🏁\n")
print(racepace_2024[["Driver", "PredictedRaceTime (s)"]])

# Evaluate Model
y_pred = model.predict(X_test)
print(f"\n🔍 Model Error (MAE): {mean_absolute_error(y_test, y_pred):.2f} seconds")
# Rank drivers by predicted race time
# qualifying_2025 = qualifying_2025.sort_values(by="PredictedRaceTime (s)")

# # Print final predictions
# print("\n🏁 Predicted 2025 Chinese GP Winner 🏁\n")
# print(qualifying_2025[["Driver", "PredictedRaceTime (s)"]])

# # Evaluate Model
# y_pred = model.predict(X_test)
# print(f"\n🔍 Model Error (MAE): {mean_absolute_error(y_test, y_pred):.2f} seconds")